In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

/home/lwd/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# One hot encoding for each char in 'hello'
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

In [3]:
with tf.variable_scope('one_cell') as scope:
    #One cell RNN input_dimension(4) > output_dimension(2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype = np.float32)
    pp.pprint(x_data)
    outputs, __states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[-0.19180335, -0.20729323]]], dtype=float32)


In [4]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[-0.21681549,  0.107999  ],
        [ 0.13126054, -0.66313225],
        [ 0.07383975,  0.260234  ],
        [-0.44577998,  0.57388037],
        [-0.61432111,  0.37480065]]], dtype=float32)


In [5]:
with tf.variable_scope('3_batches') as socpe:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)   
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
    outputs, _stats = tf.nn.dynamic_rnn(
        cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.10205432, -0.10704225],
        [ 0.05542676,  0.056013  ],
        [-0.07504582,  0.05161975],
        [-0.15610306,  0.03271016],
        [-0.01430461, -0.04531785]],

       [[-0.00358438,  0.12237688],
        [ 0.10282844,  0.02416404],
        [-0.04606458,  0.03290601],
        [-0.13857228,  0.01826296],
        [-0.19378875, -0.00334   ]],

       [[-0.11062634, -0.00725166],
        [-0.17678702, -0.02380757],
        [-0.0998603 ,  0.09164002],
        [-0.06348366,  0.18677522],
        [-0.1528257 ,

In [6]:
with tf.variable_scope('3_batches_dynamic_lenght') as socpe:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)   
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
    outputs, _stats = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length = [5,3,4] ,dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.08248758, -0.03244053],
        [-0.09800835,  0.01116376],
        [-0.113546  ,  0.09895334],
        [-0.0784886 ,  0.14840914],
        [-0.12572694,  0.24575712]],

       [[-0.04625628,  0.02729995],
        [-0.12164647,  0.1670486 ],
        [-0.06057489,  0.16548307],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.02569738,  0.09368276],
        [ 0.06188685,  0.14794043],
        [-0.00690745,  0.10039393],
        [-0.04744074,  0.09567757],
        [ 0.        ,

In [7]:
with tf.variable_scope('initial_state') as socpe:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)   
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units = hidden_size, 
                             state_is_tuple = True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _stats = tf.nn.dynamic_rnn(
        cell, x_data, initial_state = initial_state ,dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.00076382,  0.00491455],
        [-0.12414705, -0.0571281 ],
        [ 0.05406383, -0.09039623],
        [ 0.1763984 , -0.10316552],
        [-0.02516459, -0.09045175]],

       [[-0.12345655, -0.06083148],
        [-0.16721568, -0.08382386],
        [-0.05208393, -0.12483924],
        [ 0.10311237, -0.13741909],
        [ 0.20470241, -0.13859901]],

       [[ 0.14744291, -0.03578108],
        [ 0.23831779, -0.05468043],
        [ 0.04555801, -0.08773473],
        [-0.09310991, -0.12859485],
        [ 0.0720132 ,

In [8]:
#Create input data
batch_size = 3
squence_length = 5
input_dimension = 3
x_data = np.arange(45, dtype = np.float32).reshape(batch_size,
                                                   squence_length,input_dimension)
pp.pprint(x_data)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [9]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state = initial_state, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ -1.31519079e-01,   1.25343472e-01,  -2.02241570e-01,
          -2.23539975e-02,  -4.93601374e-02],
        [ -2.01150984e-01,   1.49005100e-01,  -3.75695169e-01,
          -5.50462008e-02,  -7.75760189e-02],
        [ -2.14666620e-01,   9.09091979e-02,  -2.91195661e-01,
          -2.12572161e-02,  -3.77552696e-02],
        [ -2.06931993e-01,   4.50092331e-02,   9.50919986e-02,
          -5.54535724e-03,  -9.77409072e-03],
        [ -1.81418732e-01,   2.13158280e-02,   5.85605741e-01,
          -1.25720329e-03,  -1.43059844e-03]],

       [[ -1.66807756e-01,   5.67435799e-03,   6.13698959e-01,
          -2.11841587e-04,  -1.27079411e-05],
        [ -1.40293434e-01,   3.87899484e-03,   9.10776079e-01,
          -6.09784001e-05,  -2.03899299e-06],
        [ -1.14576072e-01,   1.80072524e-03,   9.83755529e-01,
          -1.39145795e-05,  -2.28616159e-07],
        [ -1.01652294e-01,   7.62064999e-04,   9.97309268e-01,
          -3.20973913e-06,  -2.61133479e-08],
        [ -9.3258

In [10]:
# Create input data for Multi RNN
batch_size=3
sequence_length=5
input_dim=5

x_data = np.arange(75, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  

array([[[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [ 15.,  16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.,  24.]],

       [[ 25.,  26.,  27.,  28.,  29.],
        [ 30.,  31.,  32.,  33.,  34.],
        [ 35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.],
        [ 45.,  46.,  47.,  48.,  49.]],

       [[ 50.,  51.,  52.,  53.,  54.],
        [ 55.,  56.,  57.,  58.,  59.],
        [ 60.,  61.,  62.,  63.,  64.],
        [ 65.,  66.,  67.,  68.,  69.],
        [ 70.,  71.,  72.,  73.,  74.]]], dtype=float32)


In [11]:
with tf.variable_scope('MultilRNNcell') as scope:
    # make Rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] *3, state_is_tuple=True) #3 layers
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

dynamic rnn:  Tensor("MultilRNNcell/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[-0.00667916,  0.00420376,  0.00394077,  0.00467552, -0.00514032],
        [-0.01528698,  0.00839052,  0.00493819,  0.01102942, -0.01194896],
        [-0.0237998 ,  0.0118334 ,  0.00156359,  0.01759532, -0.01940313],
        [-0.03122844,  0.01443943, -0.00491179,  0.0232063 , -0.02684363],
        [-0.03692808,  0.01616615, -0.01283519,  0.0270188 , -0.03355911]],

       [[-0.00253134,  0.00186225, -0.0008368 ,  0.00201392, -0.00175845],
        [-0.00821538,  0.00567645, -0.00335814,  0.00701161, -0.00591092],
        [-0.01617474,  0.0104507 , -0.00775799,  0.01428726, -0.01225631],
        [-0.02502021,  0.01506479, -0.01345836,  0.02223935, -0.02009481],
        [-0.03356005,  0.01878211, -0.01964495,  0.02940781, -0.02852659]],

       [[-0.00447139,  0.00287916,  0.0001792 ,  0.00385581, -0.00286437],
        [-0.01398787,  0.00819651, -0.0008029 ,  0.01257607, -0.00937129],
        [-

In [12]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ -4.60960537e-01,   3.10888812e-02,  -8.50511808e-03,
          -1.99292824e-02,   4.68627930e-01],
        [ -8.01418662e-01,  -3.23275873e-03,  -1.43056721e-04,
          -8.27400107e-03,   9.12535667e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -7.59738266e-01,  -1.06784626e-07,   7.05213969e-19,
           4.59566400e-06,   7.61593878e-01],
        [ -9.62434590e-01,  -8.74642847e-09,   3.20500302e-22,
           4.63979995e-06,   9.64025676e-01],
        [ -9.94313657e-01,  -6.39845676e-10,   1.57759991e-25,
           4.23168967e-06,   9.95054245e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.

*Seq2Seq Exercise

In [15]:
#flattern based softmax
hidden_size = 3
squence_length = 5
batch_size = 3
num_classes = 5

pp.pprint(x_data)
x_data = x_data.reshape(-1,hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype = np.float32).reshape(hidden_size,
                                                     num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes)
# batch, seq, class
pp.pprint(outputs)

array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.,  34.,  35.],
       [ 36.,  37.,  38.],
       [ 39.,  40.,  41.],
       [ 42.,  43.,  44.],
       [ 45.,  46.,  47.],
       [ 48.,  49.,  50.],
       [ 51.,  52.,  53.],
       [ 54.,  55.,  56.],
       [ 57.,  58.,  59.],
       [ 60.,  61.,  62.],
       [ 63.,  64.,  65.],
       [ 66.,  67.,  68.],
       [ 69.,  70.,  71.],
       [ 72.,  73.,  74.]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.

In [22]:
# [batch_size, sequence_length]
y_data = tf.constant([[1,1,1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1,1,1]], dtype =tf.float32)
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = prediction, 
                                                targets = y_data, weights = weights )
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.596759


In [24]:
# [batch_size, sequence_length]
y_data = tf.constant([[1,1,1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3,0.7],[0.3,0.7],[0.3,0.7]]], dtype = tf.float32)
prediction2 = tf.constant([[[0.1,0.9],[0.1,0.9],[0.1,0.9]]], dtype = tf.float32)
prediction3 = tf.constant([[[1,0],[1,0],[1,0]]], dtype = tf.float32)
prediction4 = tf.constant([[[0,1],[0,1],[0,1]]], dtype = tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction4, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
     "Loss2: ", sequence_loss2.eval(),
     "Loss3: ", sequence_loss3.eval(),
     "Loss4: ", sequence_loss4.eval())

Loss1:  0.513015 Loss2:  0.371101 Loss3:  1.31326 Loss4:  0.313262
